In [3]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

In [4]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [5]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [6]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [7]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [8]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [9]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

FileNotFoundError: [Errno 2] No such file or directory: 'Map_Data/precinct-assignments-congress-nj.csv'

### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [ ]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [ ]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [ ]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [ ]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(12, nj_current_assignment, nj_contiguity)))

In [ ]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

In [ ]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West. 
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [ ]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

In [ ]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    

In [ ]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

# Fair Map

Grading Rubric
Maps produced **(2 maps, 6 pts per map)**
 - Maps in correct CSV format **(2pts)**
 - Districts are contigous **(1pts)**
 - Population is balanced **(1pts)**
 - Map is optimized for fairness/unfairness - description of how fairness/unfairness is computed is given in the report **(2pts)**
 
Report **(6 points)**
- Screenshots of the maps in DRA are included in the report **(1pts)**
- In depth description and analysis of each map **(4pts)**
    - What are their advantages?
    - What are their drawbacks?
    - Who is advantaged or penalized by the map?
    - Would the map be a reasonable option for adoption by the legislature?
- How would you imrpove your maps if you had more time? or, what other approach would you have tried. **(1pt)**

Code provided **(2 points)**


## From Existing Map - Flip Step
random based strategy similar to one in recitation_redistricting notebook

In [ ]:
def flip_precincts(current_map, district_from, district_to, num_flip, contiguity_data, precinct_data):
    border_precincts = []
    current_list = current_map[current_map['District']==district_from]['GEOID20']
    for precinct in current_list:
            neighbors = ast.literal_eval(
                contiguity_data[contiguity_data['Precinct'] == precinct]['Neighbors'].values.tolist()[0])
            for neighbor in neighbors:
                if current_map.loc[current_map['GEOID20'] == neighbor, 'District'].values.tolist()[
                    0] == district_to:
                    border_precincts.append(precinct)
                    
    flipped_precincts = np.random.choice(border_precincts, num_flip)
    for flip in flipped_precincts:
        current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_to
        # check if we broke contiguity and revert the flip if we did
        if (not isDistrictContiguous(district_from, current_map, contiguity_data) or not isDistrictContiguous(district_to, current_map, contiguity_data)):
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Contiguity broken " + flip)
        else:
            print("Went through " + flip)

In [ ]:
nj_flipstep_assignment = nj_current_assignment.copy()

# Simplistic Approach

flip_precincts(nj_flipstep_assignment, 1, 2, 8, nj_contiguity, nj_precinct_data)
flip_precincts(nj_flipstep_assignment, 2, 1, 8, nj_contiguity, nj_precinct_data)

flip_precincts(nj_flipstep_assignment, 2, 3, 8, nj_contiguity, nj_precinct_data)
flip_precincts(nj_flipstep_assignment, 3, 2, 8, nj_contiguity, nj_precinct_data)

for district in range(1, 12):
  contiguous = isDistrictContiguous(district, nj_flipstep_assignment, nj_contiguity)
  print("District", district, "Contiguous?", contiguous)

nj_flipstep_assignment.to_csv('Recitation maps/test.csv',index=False) 

In [ ]:
# previous code not functionl
#import random

# nj_flip_assignment = nj_current_assignment.copy()

# D1_Border_precincts = [] #NJ has 12 districts, and there should be 6361 precincts (rows)
# D1_list = nj_flip_assignment[nj_flip_assignment['District']==1]['GEOID20'].tolist()


# for precinct in D1_list: 
#     neighbors = nj_contiguity[nj_contiguity['GEOID20']==precinct]['Neighbor'].tolist()
#     for neighbor in neighbors:
#         if nj_flip_assignment[nj_flip_assignment['GEOID20']==neighbor]['District'].tolist()[0] == 2:
#             #if one of the neighbor is in D2, then this is a border district
#             D1_Border_precincts.append(precinct)
#             break

# # calculate percenage of total number of precincts
# num_precincts_to_flip = int(len(D1_list) * 0.15)

# for i in range(10):
#     precinct_to_flip = np.random.choice(D1_Border_precincts)
#     nj_flip_assignment.loc[nj_flip_assignment['GEOID20']==precinct_to_flip, 'District'] = 2
#     if not isDistrictContiguous(nj_flip_assignment, 1) or not isDistrictContiguous(nj_flip_assignment, 2):
#         nj_flip_assignment.loc[nj_flip_assignment['GEOID20']==precinct_to_flip, 'District'] = 1
#         print("Contiguity was broken, precinct was reassigned back to D1")


In [ ]:
# nj_flipstep_assignment = nj_current_assignment.copy()

## Simplistic Approach 
## randomly select certain number D1 precincts that border D2 and assign them to D2


# THIS CODE WORKS BUT IM GONNA COMMENT IT OUT FOR NOW
# D1_border_precincts = []

# D1_list = nj_current_assignment[nj_current_assignment['District']==1]['GEOID20'] 
# for precinct in D1_list:
#   neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
#   for neighbor in neighbors:
#     if nj_current_assignment.loc[nj_current_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==2:
#       D1_border_precincts.append(precinct)
#       break

# flipped_precincts = np.random.choice(D1_border_precincts,8)
# for flip in flipped_precincts:
#     nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#     # check if we broke contiguity and revert the flip if we did
#     if(isDistrictContiguous(1,nj_flipstep_assignment,nj_contiguity) is False or isDistrictContiguous(2,nj_flipstep_assignment,nj_contiguity) is False):
#         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#         print("Contiguity broken " + flip)
#     else:
#         populations = getDistrictPopulations(nj_flipstep_assignment,nj_precinct_data, 12)
#         pop1 = populations[1]
#         pop2 = populations[2]
#         if (abs(pop1 - pop2) / (pop1 + pop2) / 2) > 0.05:
#             nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#             print("Population deviation too high " + flip)

# D2_border_precincts = []

# D2_list = nj_current_assignment[nj_current_assignment['District']==2]['GEOID20'] 
# for precinct in D2_list:
#   neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
#   for neighbor in neighbors:
#     if nj_current_assignment.loc[nj_current_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==2:
#       D2_border_precincts.append(precinct)
#       break

# flipped_precincts = np.random.choice(D2_border_precincts,8)
# for flip in flipped_precincts:
#     nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#     # check if we broke contiguity and revert the flip if we did
#     if(isDistrictContiguous(1,nj_flipstep_assignment,nj_contiguity) is False or isDistrictContiguous(2,nj_flipstep_assignment,nj_contiguity) is False):
#         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#         print("Contiguity broken " + flip)
#     else:
#         populations = getDistrictPopulations(nj_flipstep_assignment,nj_precinct_data, 12)
#         pop1 = populations[1]
#         pop2 = populations[2]
#         if (abs(pop1 - pop2) / (pop1 + pop2) / 2) > 0.05:
#             nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#             print("Population deviation too high " + flip)

# Now 2 and 3
# D2_border_precincts = []

# D2_list = nj_flipstep_assignment[nj_flipstep_assignment['District']==2]['GEOID20'] 
# for precinct in D2_list:
#   neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
#   for neighbor in neighbors:
#     if nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==2:
#       D2_border_precincts.append(precinct)
#       break

# flipped_precincts = np.random.choice(D2_border_precincts,8)
# for flip in flipped_precincts:
#     nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=3
#     # check if we broke contiguity and revert the flip if we did
#     if(isDistrictContiguous(1,nj_flipstep_assignment,nj_contiguity) is False or isDistrictContiguous(2,nj_flipstep_assignment,nj_contiguity) is False):
#         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#         print("Contiguity broken " + flip)
#     else:
#         populations = getDistrictPopulations(nj_flipstep_assignment,nj_precinct_data, 12)
#         pop1 = populations[1]
#         pop2 = populations[2]
#         if (abs(pop1 - pop2) / (pop1 + pop2) / 2) > 0.05:
#             nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=3
#             print("Population deviation too high " + flip)

        
# Sample precints and flip them UNLESS it breaks contiguity
# flipped_precincts = np.random.choice(D1_border_precincts,5)

# for flip in flipped_precincts:
#     nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#     #check if we broke contiguity and revert the flip if we did
#     if(isDistrictContiguous(1,nj_flipstep_assignment,nj_contiguity) is False or isDistrictContiguous(2,nj_flipstep_assignment,nj_contiguity) is False):
#         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#         print("Contiguity broken " + flip)
    # else:
    #     populations = getDistrictPopulations(nj_flipstep_assignment,nj_precinct_data, 5)
    #     pop1 = populations[1]
    #     pop2 = populations[2]
    #     if (abs(pop1 - pop2) / (pop1 + pop2) / 2) > 0.05:
    #         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
    #         print("Population deviation too high " + flip)

# First we randomly select 5 DIFFERENT D2 precincts that border D1 and assign them to D1
# D2_border_precincts = []
# D2_list = nj_longlat_assignment[nj_longlat_assignment['District']==2]['GEOID20']
# for precinct in D2_list:
#     neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
#     for neighbor in neighbors:
#         if nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==1 and precinct not in flipped_precincts:
#             #if one of the neighbor is in D2, then this is a border district. We do not want to re-flip a district we just flipped
#             D2_border_precincts.append(precinct)
#             break

#Sample 5 and flip them UNLESS it breaks contiguity
# if len(D2_border_precincts) >= 5:
#     flipped_precincts = np.random.choice(D2_border_precincts,5)
# else:
#     print("Not enough border precincts to flip")
#     flipped_precincts = []

# print("Number of D2 border precincts:", len(D2_border_precincts))
# print("D2 border precincts:", D2_border_precincts)


# for flip in flipped_precincts:
#     nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=1
#     #check if we broke contiguity and revert the flip if we did
#     if(isDistrictContiguous(1,nj_flipstep_assignment,nj_contiguity) is False or isDistrictContiguous(2,nj_flipstep_assignment,nj_contiguity) is False):
#         nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#         print("Contiguity broken " + flip)
#     else:
#         populations = getDistrictPopulations(nj_flipstep_assignment,nj_precinct_data, 12)
#         pop1 = populations[1]
#         pop2 = populations[2]
#         if (abs(pop1 - pop2) / (pop1 + pop2) / 2) > 0.05:
#             nj_flipstep_assignment.loc[nj_flipstep_assignment['GEOID20']==flip,'District']=2
#             print("Population deviation too high " + flip)


# for district in range(1, 12):
#   contiguous = isDistrictContiguous(district, nj_flipstep_assignment, nj_contiguity)
#   print("District", district, "Contiguous?", contiguous)

# print(len(D1_border_precincts))
# print(len(D2_border_precincts))
# print(flipped_precincts) 
# print(len(set(flipped_precincts))) # check unique

# nj_flipstep_assignment.to_csv('Recitation maps/test.csv',index=False)  

# Unfair Map
map should be designed to give a party/community/area some advantage (unfair map) using some creative gerrymandering techniques. The less likely the map would be to be thrown out of courts for obvious gerrymandering, the better.